# Compare FSE/VarBugs

This portion of the notebook allows you to compare the results between the sampling-based baseline and the desugared analysis results. This will print out number of results from analyzing desugared code, the number of baselines, and the overlap between them.

To use this portion of the notebook, please set "result_location" to the location of the desugared results, and "baseline_location" to the location of the sampling-based baseline results.

In [17]:
#Set Baseline and Experiment
result_location = "/Users/austin/git/Sugarlyzer/results.json"
baseline_location = "/Users/austin/git/Sugarlyzer/baseline.json"

import json
from json import JSONEncoder

def _default(self, obj):
    return getattr(obj.__class__, "to_json", _default.default)(obj)

_default.default = JSONEncoder().default
JSONEncoder.default = _default

import logging
import copy
from pathlib import Path
from typing import Tuple

with open(baseline_location) as f:
    baselines = json.load(f)

with open(result_location) as f:
    experimental_results = json.load(f)

lonely_baselines = copy.deepcopy(baselines)
lonely_experimental_results = copy.deepcopy(experimental_results)

class IntRange:
    def __init__(self, lower_bound_inclusive, upper_bound_exclusive):
        self.lower_bound_inclusive = lower_bound_inclusive
        self.upper_bound_exclusive = upper_bound_exclusive

    def __contains__(self, item):
        return isinstance(item, int) and (self.lower_bound_inclusive <= item < self.upper_bound_exclusive)

    def __repr__(self):
        return f"IntRange({self.lower_bound_inclusive, self.upper_bound_exclusive})"

    def __str__(self):
        return f"[{self.lower_bound_inclusive}:{self.upper_bound_exclusive})"

    def to_json(self):
        return str(self)


for e in experimental_results:
    toks = e['original_line'].split(':')
    try:
        e['original_line'] = IntRange(int(toks[0]), int(toks[1]) + 1)
    except Exception as ex:
        e['original_line'] = []
    #print('\t'.join(["experimental", *[str(s) for s in e.values()]]).replace("\n", ""))

    if e['function_line_range'] == 'ERROR':
        e['function_line_range'] = []
    else:
        toks = e['function_line_range'].split(':')
        try:
            e['function_line_range'] = IntRange(int(toks[1]), int(toks[2]) + 1)
        except Exception as ex:
            logging.exception(f"e was {e}")
    e['presence_condition'] = str(e['presence_condition'])

import tqdm

def match_stats(baseline_result: dict, experimental_result: dict) -> Tuple:
    """
    Returns a vector of different match information.
    (a, b, c)
    a = True iff baseline and experimental have the same line number, message, and file.
    b = True iff baseline and experimental have the same message, file, and baseline is within experimental's function scope.
    c = True iff baseline's configuration is compatible with experimental's presence condition.
    """

    a = (baseline_result['sanitized_message'].lstrip().rstrip() == experimental_result['sanitized_message'].lstrip().rstrip() and \
         baseline_result['input_line'] in experimental_result['original_line'] and\
         baseline_result['input_file'].split('.')[0] == experimental_result['input_file'].split('.')[0])

    b = (baseline_result['sanitized_message'].lstrip().rstrip() == experimental_result['sanitized_message'].lstrip().rstrip() and \
         baseline_result['input_line'] in experimental_result['function_line_range'] and\
         baseline_result['input_file'].split('.')[0] == experimental_result['input_file'].split('.')[0])

    c = False

    if experimental_result['feasible'] and 'Or(None' not in experimental_result['presence_condition'] and experimental_result['presence_condition'] not in ['Or(None)', 'None'] and (a or b):  # Don't bother doing this expensive step when the file and line number are different.
        baseline_var_mapping = {}
        for var in baseline_result['configuration']:
            if type(var) is str:
                if var.startswith('DEF'):
                    baseline_var_mapping[re.sub(r"^(DEF_.*)", r"\1", var)] = True
                elif var.startswith('UNDEF'):
                    baseline_var_mapping[re.sub(r"^UN(DEF_.*)", r"\1", var)] = False
                else:
                    raise RuntimeError(f"Don't know how to handle variable {var}")

        s = Solver()
        for var, val in baseline_var_mapping.items():
            var = Bool(var)
            if val:
                s.add(var)
            else:
                s.add(Not(var))

        for mat in re.findall("DEF_[a-zA-Z0-9_]+", experimental_result['presence_condition']):
            exec(f"{mat} = Bool('{mat}')")
           
        for mat in re.findall("USE_[a-zA-Z0-9_]+", experimental_result['presence_condition']):
            exec(f"{mat} = Int('{mat}')")

        while True:
            try:
                s.add(eval(experimental_result['presence_condition']))  # TODO Definitely need to do more transformation here.
                break
            except NameError as ne:
                var = re.search("name '(.*)' is not defined", str(ne))
                exec(f"{var.group(1)} = Int('{var.group(1)}')")
        c = s.check() == sat
    return a, b, c

def tupleize(func, args): return func(*args), tuple(args)

summary = {}

# Note that results depend on the order of keys in this dictionary, because once we find a match_stats for one level we do not keep searching for the next.
#  E.g., for a given report, we will first look for results with which it has a (True, True, True) report. If it has one, we do not continue searching for
#  matches for (False, True, True), (True, False, True), etc.
result_hierarchy = {(True, True, True): 0, (False, True, True): 0, (True, False, True): 0, (True, True, False): 0, (False, True, False): 0, (False, False, True): 0, (True, False, False): 0, (False, False, False): 0}

report = []
for b in tqdm.tqdm(baselines):
    # Results are (baseline, desugared, match tuple)
    results = [(b, e, match_stats(b, e)) for e in experimental_results]
    found = False
    for r in result_hierarchy.keys():
        for res in results:
            if res[2] == r:
                found = True
                result_hierarchy[r] += 1
                # -----
                # Here is where you compile information about any specific reports you need. This block of code
                # iterates through all baselines and finds the highest level of matching that is available.
                # So, for example, if you wanted to collect all of the unmatched originals, you would uncomment out this line of code:
                #
                if (r != (True, True, True) and r != (False, True, True)):
                    report.append(res[0])
                break # DO NOT DELETE THE BREAK!
        if found:
            break

print(f"Number of baseline results: {len(baselines)}")
print(f"Number of desugared results: {len(experimental_results)}")
print(f"Number of feasible desugared results: {len(list(filter(lambda e: e['feasible'], experimental_results)))}")
print(f"Number of exact matched baselines: {result_hierarchy[(True, True, True)]}")
print(f"Number of partially matched baselines: {result_hierarchy[False, True, True]}")
print(
    f"Number of unmatched baselines: {sum(v for k, v in result_hierarchy.items() if k not in [(True, True, True), (False, True, True)])}")

100%|██████████| 85/85 [00:00<00:00, 58743.75it/s]

Number of baseline results: 85
Number of desugared results: 21
Number of feasible desugared results: 21
Number of exact matched baselines: 0
Number of partially matched baselines: 0
Number of unmatched baselines: 85


# Time Analysis
This part of the notebook will compute the total time the analysis took, including both desugaring time and the analysis time.
This requires the log files produced by Sugarlyzer.

In [16]:
log_file = "/Users/austin/Downloads/sugarlyzerTiming/sugarlyzerRes/clang/axtls/log.txt"

import os
import sys
import re

dtime = 0
atime = 0
with open(log_file,'r') as i:
    for line in i.readlines():
        line = line.lstrip().rstrip()
        if 'Analyzing file' in line and ' took ' in line:
            res = re.search(r'took (\d+\.\d+)s',line)
            if res != None:
                atime += float(res.group(1))
            
        elif ' desugared in time:' in line:
            res = re.search(r' desugared in time:(\d+\.\d+)',line)
            if res != None:
                dtime += float(res.group(1))
            
print (f'Analysis Time: {int(atime/60)}m\nDesugaring time: {int(dtime/60)}m')

Analysis Time: 1m
Desugaring time: 12m
